In [1]:
# 1  Install & import libraries
!pip install -q google-genai tqdm

from google import genai
from google.genai import types
from pathlib import Path
import os, json, time, re, shutil, tqdm
from typing import List, Dict


In [2]:
# 2  Authenticate Gemini & mount Drive

# === Provide your API key (or export it beforehand) ===
os.environ['GEMINI_API_KEY'] = ''  

client = genai.Client()
print('✅ Gemini client initialised')

# Mount Drive (optional – comment out if running locally)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


✅ Gemini client initialised
Mounted at /content/drive


In [3]:

drive_root = Path('/content/drive/MyDrive/HotpotQA_snapshot')
work_dir   = Path('/content/llm_answer_generation')
work_dir.mkdir(parents=True, exist_ok=True)

src_file   = drive_root / 'evidence_hotpotqa_entities_with_paths.json'
dst_file   = work_dir   / src_file.name

shutil.copy2(src_file, dst_file)
print('📄 Evidence file copied to working dir')


📄 Evidence file copied to working dir


In [7]:

def load_json(path: Path) -> List[Dict]:
    with open(path, 'r') as f:
        return json.load(f)

def save_json(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        json.dump(obj, f, indent=4, ensure_ascii=False)

def _clean_answer(text) -> str:
    """Return a whitespace-collapsed answer; tolerate None/empty inputs."""
    if not isinstance(text, str):
        return ""
    text = re.sub(r"<think[^>]*?>.*?</think>", " ", text, flags=re.S)
    return re.sub(r"\s+", " ", text).strip()

def split_thoughts_and_answer(parts):
    """Separate thought summaries from the final answer (returns thoughts, answer)."""
    thoughts, answer = [], []
    for p in parts:
        if not getattr(p, 'text', None):
            continue
        if getattr(p, 'thought', False):
            thoughts.append(p.text)
        else:
            answer.append(p.text)
    return "\n".join(thoughts).strip(), " ".join(answer).strip()


In [5]:

def generate_answers(
    evidence_file: Path,
    save_file: Path,
    *,
    max_output_tokens: int = 128,
    temperature: float = 0.6,
    top_p: float = 0.95,
    thinking_budget: int = -1,   # dynamic
    pause_secs: float = 5.0,
):
    data = load_json(evidence_file)
    responses = []

    prompt_template = (
        'Given the question, its associated knowledge-graph paths, and evidence contexts below, '
        'please generate a concise, precise answer in English. The answer must strictly adhere to the following guidelines:\n'
        '- The answer should be directly relevant to the question.\n'
        '- Provide the answer in a clear, straightforward format.\n'
        '- Limit your answer to **no more than 10 words**, focusing on the essential information requested.\n'
        '- Use both the provided information and your own knowledge to answer as accurately as possible.\n\n'
        'QUESTION: {question}\n'
        'PATHS:\n{paths}\n'
        'CONTEXT:\n{context}\n'
        'ANSWER:'
    )

    none_template = (
        'Given the following question, create a concise (≤10 words) English answer using your own knowledge if needed.\n'
        'QUESTION: {question}\n'
        'ANSWER:'
    )

    for rec in tqdm.tqdm(data, total=len(data)):
        ctx_list  = rec.get('evidence', [])
        paths_list = rec.get('Final_paths') or rec.get('paths', [])

        if ctx_list or paths_list:
            ctx_block   = "\n".join(f"{i+1}: {c}" for i, c in enumerate(ctx_list))
            paths_block = "\n".join(f"{i+1}: {p}" for i, p in enumerate(paths_list))
            user_prompt = prompt_template.format(
                question = rec['question'],
                paths    = paths_block if paths_block else 'N/A',
                context  = ctx_block if ctx_block else 'N/A'
            )
        else:
            user_prompt = none_template.format(question=rec['question'])

        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=user_prompt,
            config=types.GenerateContentConfig(
                temperature=temperature,
                top_p=top_p,
                max_output_tokens=max_output_tokens,
                thinking_config=types.ThinkingConfig(
                    thinking_budget=thinking_budget,
                    include_thoughts=True
                ),
            ),
        )

        try:
            cand_container = response.candidates
            cand = cand_container[0] if isinstance(cand_container, (list, tuple)) else cand_container
            parts = cand.content.parts
            thought_text, answer_text = split_thoughts_and_answer(parts)
        except (TypeError, IndexError, AttributeError):
            thought_text = ""
            answer_text  = getattr(response, "text", "Information not available")

        short_answer = _clean_answer(answer_text)

        responses.append({
            'type':         rec.get('type'),
            'question':     rec['question'],
            'answer':       rec.get('answer'),      # ground-truth answer untouched
            'thoughts':     thought_text,
            'response':     answer_text,            
            'short_answer': short_answer,          
        })

        save_json(responses, save_file)  
        time.sleep(pause_secs)           

    print(f"Finished – saved to {save_file}")
    return responses


In [8]:
# 6  Run generation on HotpotQA

hotpot_out = drive_root / 'hotpotqa_answers_gemini.json'

_ = generate_answers(
        evidence_file = work_dir / 'evidence_hotpotqa_entities_with_paths.json',
        save_file     = hotpot_out,
        max_output_tokens = 1024,
        temperature       = 0.6,
        top_p             = 0.95,
        thinking_budget   = -1,   # dynamic
        pause_secs        = 10,
)

print('✅ All done')


100%|██████████| 100/100 [24:31<00:00, 14.72s/it]

Finished – saved to /content/drive/MyDrive/HotpotQA_snapshot/hotpotqa_answers_gemini.json
✅ All done
